In [ ]:
import pandas as pd 
import scsims
from sklearn.preprocessing import LabelEncoder
import sys
import anndata as an
import scanpy as sp
import h5py

sys.path.append('../src')
sys.path.append('../tests')

from models.lib.lightning_train import DataModule, generate_trainer
from models.lib.data import *
from models.lib.neural import *
from pytorch_lightning.loggers import WandbLogger
from torchmetrics.functional import *
import pandas as pd
from scipy.sparse import csr_matrix
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from bigcsv.bigcsv import experimental_to_h5ad

Let's do some simple preprocessing for training, and then test our library

In [ ]:
# le = LabelEncoder()

# labels = pd.read_csv('../data/retina/retina_labels.tsv', sep='\t')
# labels = labels[labels['CellType'].isin(['retinal rod cell type B', 'retinal rod cell type A'])]
# labels = labels[labels['CellType'] != 'unannotated']
# labels['class_label'] = le.fit_transform(labels['CellType'])

# labels.index.name = 'cell'
# # labels = labels.iloc[0:5000, :]
# labels.to_csv('../data/retina/retina_labels_numeric.csv')

# # label_df = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
# # label_df

# # labels.to_csv('../data/retina/retina_labels_numeric.csv')

# labels = pd.read_csv('../data/retina/retina_labels_numeric.csv')
# labels.loc[:, 'class_label']

In [ ]:
# labels = pd.read_csv('../data/retina/raw_labels.tsv', sep='\t')
# corrected = pd.read_csv('../data/retina/retina_labels_numeric.csv')

In [4]:
# from sklearn.model_selection import train_test_split
# trainsplit, valsplit = train_test_split(current_labels, stratify=current_labels, random_state=42)
# trainsplit

In [5]:
corrected = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
print(corrected.shape)
# current_labels = corrected['CellType']

# trainsplit, valsplit = train_test_split(current_labels, stratify=current_labels, random_state=42)
# trainsplit

(16446, 8)


In [6]:
module = DataModule(
    datafiles=['../data/retina/retina_T.h5ad'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    index_col='cell',
    batch_size=16,
    num_workers=8,
    shuffle=True,
    drop_last=True,
    normalize=True,
)

module.setup()

Creating train/val/test DataLoaders...


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Done, continuing to training.
Calculating weights


In [7]:
!mkdir checkpoints

mkdir: checkpoints: File exists


In [8]:
import pytorch_lightning as pl

lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')
checkpoint = pl.callbacks.ModelCheckpoint(
    dirpath='checkpoints', 
    filename='{epoch}-{weighted_val_accuracy}'
)
# pruner = pl.callbacks.ModelPruning()
progressbar = pl.callbacks.RichProgressBar()

In [9]:
tabnetmodel = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
        'weight_decay': 0,
    },
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau,
        'factor': 0.001,
    },
    metrics={
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    },
)

wandb_logger = WandbLogger(
    project=f"Retina Model",
    name='local-retina-model'
)

early_stop = EarlyStopping(
    monitor="weighted_val_accuracy", 
    min_delta=0.00, 
    patience=3,
    verbose=False, 
    mode="max"
)

trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=[early_stop, lr_monitor, checkpoint],
    max_epochs=100,
)

Initializing network
Initializing explain matrix


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(tabnetmodel, datamodule=module)

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)


┏━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name    ┃ Type   ┃ Params ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ 0 │ network │ TabNet │  2.5 M │
└───┴─────────┴────────┴────────┘

Trainable params: 2.5 M                                                                      
Non-trainable params: 0                                                                      
Total params: 2.5 M                                                                          
Total estimated model params size (MB): 9

Output()

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning
/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is 
strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning
/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have 
many workers which may be a bottleneck. Consider increasing the value of the `num_workers` 
argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to 
improve performance.
  rank_zero_warn(

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:623: UserWarning: Checkpoint directory /Users/julian/Documents/Projects/organoid-classification/notebooks/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning
/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have 
many workers which may be a bottleneck. Consider increasing the value of the `num_workers` 
argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to 
improve performance.
  rank_zero_warn(

In [10]:
pd.read_csv('../data/retina/fovea_for_test_labels.tsv', sep='\t')['celltype'].unique()

array(['Unknown', 'Glial-F1', 'Cones-Fov', 'Bipolar-1', 'Horizontal',
       'Endothelial', 'Pericytes', 'Glial-P3', 'Bipolar-2', 'Rods-1',
       'RGC', 'Rods-2', 'Glial-F2', 'Amacrine', 'Microglia', 'Glial-P2',
       'Glial-P1', 'Cones-Periph'], dtype=object)

In [13]:
pd.read_csv('../data/retina/retina_labels_numeric.csv')['CellType'].unique()

array(['retinal rod cell type A', 'retinal rod cell type B',
       'retinal bipolar neuron type B', 'retinal rod cell type C',
       'unspecified', 'retinal bipolar neuron type C', 'Muller cell',
       'retinal cone cell', 'retinal bipolar neuron type A',
       'amacrine cell', 'retinal bipolar neuron type D',
       'retinal ganglion cell', 'microglial cell'], dtype=object)